In [1]:
import numpy as np
import MarketDataFetcher
import asyncio
import nest_asyncio
import BinanceBackTest
import utils
import os

nest_asyncio.apply()

In [2]:
symbols = ['BTCUSDT', 'ADAUSDT', 'XRPUSDT']
intervals = ['1m', '3m', '5m', '1h']

BBT = BinanceBackTest.DataManager(symbols=symbols,
                                  intervals=intervals,
                                  start_date='2024-8-1 00:00:00',
                                  end_date='2024-11-03 23:59:59')
path = os.path.join(os.path.dirname(os.getcwd()), 'DataStore/kline_data.json')
kline_data = utils._load_json(file_path=path)
# utils._load_to_json(file_path = path)
# kline_data = asyncio.run(BBT.generate_kline_interval_data())#(save=True))

In [3]:
closing_sync_data = BBT.generate_kline_closing_sync(kline_data)

[[1.72243800e+12 6.64100000e+04 6.64292000e+04 ... 5.81860000e+01
  3.86412686e+06 0.00000000e+00]
 [1.72243806e+12 6.63647000e+04 6.63718000e+04 ... 6.88700000e+01
  4.56886406e+06 0.00000000e+00]
 [1.72243812e+12 6.63558000e+04 6.63726000e+04 ... 6.32240000e+01
  4.19476136e+06 0.00000000e+00]
 ...
 [1.73065782e+12 6.83097000e+04 6.83097000e+04 ... 3.23910000e+01
  2.21187685e+06 0.00000000e+00]
 [1.73065788e+12 6.82680000e+04 6.82919000e+04 ... 9.73470000e+01
  6.64650865e+06 0.00000000e+00]
 [1.73065794e+12 6.82768000e+04 6.82795000e+04 ... 4.05610000e+01
  2.76899522e+06 0.00000000e+00]]
 open 3m - 1722438180000.0
 close 3m - 1722438359999.0
 open 5m - 1722438600000.0
 close 5m - 1722438899999.0
 open 1h - 1722448800000.0
 close 1h - 1722452399999.0
 open 3m - 1722438180000.0
 close 3m - 1722438359999.0
 open 5m - 1722438600000.0
 close 5m - 1722438899999.0
 open 1h - 1722448800000.0
 close 1h - 1722452399999.0
 open 3m - 1722438180000.0
 close 3m - 1722438359999.0
 open 5m - 1722

In [4]:
path = os.path.join(os.path.dirname(os.getcwd()), 'DataStore/kline_data.json')

kline_data = utils._load_json(path)

def convert_to_container(kline_data):
    container_data = utils.DataContainer()
    
    map_symbol = {}
    map_interval = {}
    
    symbols = list(kline_data.keys())
    intervals = list(kline_data[symbols[0]].keys())
    
    for idx_i, interval in enumerate(intervals):
        map_interval[interval] = idx_i
    
        dummy_data = []
        
        for idx_s, symbol in enumerate(symbols):
            map_symbol[symbol] = idx_s
    
            target_data = kline_data[symbol][interval]
            dummy_data.append(target_data)
    
        dummy_data = np.array(dummy_data)
        container_data.set_data(data_name = f'interval_{interval}', data=dummy_data)
    return map_symbol, map_interval, container_data

In [5]:
dummy = BBT.convert_kline_data_array(kline_data=kline_data)

AttributeError: 'DataManager' object has no attribute 'convert_kline_data_array'

In [6]:
a, b, c = convert_to_container(kline_data)

In [7]:
c.get_data('interval_3m')

array([[['1722438000000', '66410.00', '66429.20', ..., '190.280',
         '12627752.27160', '0'],
        ['1722438180000', '66363.50', '66464.10', ..., '174.522',
         '11588638.51030', '0'],
        ['1722438360000', '66421.10', '66424.70', ..., '111.528',
         '7403106.78550', '0'],
        ...,
        ['1730739060000', '67774.00', '67867.40', ..., '517.712',
         '35098109.92350', '0'],
        ['1730739240000', '67776.70', '67781.90', ..., '642.677',
         '43518794.66970', '0'],
        ['1730739420000', '67720.00', '67745.60', ..., '968.809',
         '65563554.65000', '0']],

       [['1722438000000', '0.40310', '0.40340', ..., '517927',
         '208616.59790', '0'],
        ['1722438180000', '0.40290', '0.40390', ..., '445865',
         '179876.48840', '0'],
        ['1722438360000', '0.40310', '0.40320', ..., '167165',
         '67289.68490', '0'],
        ...,
        ['1730739060000', '0.32550', '0.32590', ..., '230397',
         '75047.16650', '0'],
     

In [8]:
symbol = 'BTCUSDT'
target_interval = '3m'

symbols = list(kline_data.keys())
interval = list(kline_data[symbols[0]].keys())

result = {}
empty_data = []
for i in kline_data[symbol][target_interval]:
    open_timestamp = i[0]
    close_timestamp = i[6]
    condition = np.where((
        kline_data[symbol]['1m'][:,0]>=open_timestamp)&
    (kline_data[symbol]['1m'][:,6]<=close_timestamp))[0]
    for n in condition:
        if n == condition[-1]:
            empty_data.append(i)
            continue
        if n == condition[0]:
            new_data = [
                open_timestamp,
                kline_data[symbol]['1m'][n][1],
                kline_data[symbol]['1m'][n][2],
                kline_data[symbol]['1m'][n][3],
                kline_data[symbol]['1m'][n][4],
                kline_data[symbol]['1m'][n][5],
                close_timestamp,
                kline_data[symbol]['1m'][n][7],
                kline_data[symbol]['1m'][n][8],
                kline_data[symbol]['1m'][n][9],
                kline_data[symbol]['1m'][n][10],
                0,
            ]
        else:
            new_data = [
                open_timestamp,
                kline_data[symbol]['1m'][n][1],
                max(kline_data[symbol]['1m'][n][2], empty_data[-1][2]),
                min(kline_data[symbol]['1m'][n][3], empty_data[-1][3]),
                kline_data[symbol]['1m'][n][4],
                kline_data[symbol]['1m'][n][5] + empty_data[-1][5],
                close_timestamp,
                kline_data[symbol]['1m'][n][7] + empty_data[-1][7],
                kline_data[symbol]['1m'][n][8] + empty_data[-1][8],
                kline_data[symbol]['1m'][n][9] + empty_data[-1][9],
                kline_data[symbol]['1m'][n][10] + empty_data[-1][10],
                0,
            ]
        empty_data.append(new_data)

TypeError: list indices must be integers or slices, not tuple

In [9]:
n

NameError: name 'n' is not defined

In [10]:
import utils

symbol = 'BTCUSDT'
target_interval = '3m'

symbols = list(kline_data.keys())
interval = list(kline_data[symbols[0]].keys())

result = {}

# convert_data = self.convert_kline_data_array(kline_data=kline_data)
obj = utils.DataContainer()

main_data = []
for symbol, kline_data_symbol in kline_data.items():
    a_ = [kline_data[symbol][intervals[0]]]
    for interval, _ in kline_data_symbol.items():
        if interval == intervals[0]:
            continue
        d_ = []
        for i in dummy[symbol][target_interval]:
            open_timestamp = i[0]
            close_timestamp = i[6]
            condition = np.where((
                dummy[symbol]['1m'][:,0]>=open_timestamp)&
            (dummy[symbol]['1m'][:,6]<=close_timestamp))[0]
            for n in condition:
                if n == condition[-1]:
                    empty_data.append(i)
                    continue
                if n == condition[0]:
                    new_data = [
                        open_timestamp,
                        dummy[symbol]['1m'][n][1],
                        dummy[symbol]['1m'][n][2],
                        dummy[symbol]['1m'][n][3],
                        dummy[symbol]['1m'][n][4],
                        dummy[symbol]['1m'][n][5],
                        close_timestamp,
                        dummy[symbol]['1m'][n][7],
                        dummy[symbol]['1m'][n][8],
                        dummy[symbol]['1m'][n][9],
                        dummy[symbol]['1m'][n][10],
                        0,
                    ]
                else:
                    new_data = [
                        open_timestamp,
                        dummy[symbol]['1m'][n][1],
                        max(dummy[symbol]['1m'][n][2], empty_data[-1][2]),
                        min(dummy[symbol]['1m'][n][3], empty_data[-1][3]),
                        dummy[symbol]['1m'][n][4],
                        dummy[symbol]['1m'][n][5] + empty_data[-1][5],
                        close_timestamp,
                        dummy[symbol]['1m'][n][7] + empty_data[-1][7],
                        dummy[symbol]['1m'][n][8] + empty_data[-1][8],
                        dummy[symbol]['1m'][n][9] + empty_data[-1][9],
                        dummy[symbol]['1m'][n][10] + empty_data[-1][10],
                        0,
                    ]
            # obj.set_data(data_name=f'interval_{interval}', data=[new_data])
                d_.append(new_data)
        main_data.append(d_)
    # main_data = np.array(main_data)


NameError: name 'dummy' is not defined

In [ ]:
main_data

In [ ]:
symbol = 'BTCUSDT'
target_interval = '3m'

def ex(kline_data :dict):
    symbols = list(kline_data.keys())
    intervals = list(kline_data[symbols[0]].keys())

    result = {}
    
    symbol_map = {}
    interval_map = {}
    
    
    for idx_s, (symbol, kline_data_symbol) in enumerate(kline_data.items()):
        symbol_map[symbol] = idx_s
        result[symbol] = {}
        for idx_i, (interval, kline_data_interval) in enumerate(kline_data_symbol.items()):
            interval_map[interval] = idx_i
            
            if interval == intervals[0]:
                result[symbol][interval] = kline_data_interval
                continue
            
            
            empty_data = []
            for kline_data_interval in kline_data[symbol][intervals[1:]]:
                open_timestamp = kline_data_interval[0]
                close_timestamp = kline_data_interval[6]
                condition = np.where((
                    kline_data[symbol]['1m'][:,0]>=open_timestamp)&
                (kline_data[symbol]['1m'][:,6]<=close_timestamp))[0]
                for n in condition:
                    if n == condition[-1]:
                        empty_data.append(kline_data_interval)
                        continue
                    if n == condition[0]:
                        new_data = [
                            open_timestamp,
                            kline_data[symbol]['1m'][n][1],
                            kline_data[symbol]['1m'][n][2],
                            kline_data[symbol]['1m'][n][3],
                            kline_data[symbol]['1m'][n][4],
                            kline_data[symbol]['1m'][n][5],
                            close_timestamp,
                            kline_data[symbol]['1m'][n][7],
                            kline_data[symbol]['1m'][n][8],
                            kline_data[symbol]['1m'][n][9],
                            kline_data[symbol]['1m'][n][10],
                            0,
                        ]
                    else:
                        new_data = [
                            open_timestamp,
                            kline_data[symbol]['1m'][n][1],
                            max(kline_data[symbol]['1m'][n][2], empty_data[-1][2]),
                            min(kline_data[symbol]['1m'][n][3], empty_data[-1][3]),
                            dumkline_datay[symbol]['1m'][n][4],
                            kline_data[symbol]['1m'][n][5] + empty_data[-1][5],
                            close_timestamp,
                            kline_data[symbol]['1m'][n][7] + empty_data[-1][7],
                            kline_data[symbol]['1m'][n][8] + empty_data[-1][8],
                            kline_data[symbol]['1m'][n][9] + empty_data[-1][9],
                            kline_data[symbol]['1m'][n][10] + empty_data[-1][10],
                            0,
                        ]
                    empty_data.append(new_data)
            result[symbol][interval] = empty_data
            
            
        
    
    

In [ ]:
path = os.path.join(os.path.dirname(os.getcwd()), 'DataStore/kline_data.json')

kline_data = utils._load_json(path)

def convert_to_container(kline_data):
    container_data = utils.DataContainer()
    
    map_symbol = {}
    map_interval = {}
    
    symbols = list(kline_data.keys())
    intervals = list(kline_data[symbols[0]].keys())
    
    for idx_i, interval in enumerate(intervals):
        map_interval[interval] = idx_i
    
        dummy_data = []
        
        for idx_s, symbol in enumerate(symbols):
            map_symbol[symbol] = idx_s
    
            target_data = kline_data[symbol][interval]
            dummy_data.append(target_data)
    
        dummy_data = np.array(dummy_data)
        container_data.set_data(data_name = f'interval_{interval}', data=dummy_data)
    return map_symbol, map_interval, container_data